In [1029]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import platform
import seaborn as sns
import os

In [1031]:
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

print(platform.system()) # 플랫폼 확인

if platform.system() == 'Windows': # Window
    matplotlib.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    matplotlib.rc('font', family='AppleGothic')

Darwin


In [966]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [1040]:
folder = os.listdir()

file = []
for f in folder:
    if "_2022" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
# gajwa = gajwa.dropna(axis=0)

In [1041]:
geo = pd.read_csv("./geocoding.csv")
gajwa["경도"] = geo.X
gajwa["위도"] = geo.Y

gajwa = gajwa.fillna({'건축년도':gajwa['건축년도'].mode()[0]})
gajwa = gajwa.fillna({'도로명':gajwa['도로명'].mode()[0]})

# 도로조건을 정수로 변환
road=[]
for a in gajwa["도로조건"]:
    if a=="-":
        a = "12"
    road.append(int(a.split("m")[0]))
gajwa["도로조건"] = road


# 년월 column 생성
year=[]
month=[]
for d in gajwa["계약년월"]:
    day = pd.to_datetime(d, format="%Y%m")
    year.append(day.year)
    month.append(day.month)

gajwa["year"] = year
gajwa["month"] = month

built_day = [pd.to_datetime(d, format="%Y").year for d in gajwa["건축년도"]]
gajwa["건축년도"] = built_day

# 위치 column만들기
gajwa["location"] = gajwa.시군구 + " " + gajwa.도로명


#원핫인코딩으로 전월세 구분하기
gajwa = pd.get_dummies(gajwa, columns = ["전월세구분"])

# 계약년월 기준으로 전월세전환율 입력하기
rate = pd.read_csv("./rate.csv", encoding="cp949")
rate = rate.T
rate = rate[3:]

month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep


# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa["월세(만원)"][i] * 12 / (gajwa["전월세전환율"][i]/100) + gajwa["보증금(만원)"][i]) for i in range(len(gajwa))]
gajwa["전세전환가격"] = value
#gajwa["도로별 전세값평균"] = gajwa.groupby("도로명").전세전환가격.transform("mean")


# 도로명 결측치 제거
indexNames = []
for i in range(len(gajwa["도로명"])):
    if "번길" not in gajwa["도로명"][i]:
        indexNames.append(i)

gajwa.drop(indexNames, inplace=True)

gajwa

,시군구,번지,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,위도,year,month,location,전월세구분_월세,전월세구분_전세,전월세전환율,전세전환가격
0,경상남도 진주시 가좌동,1***,12,12.00,202003,5,1500,25,2007,진주대로404번길,128.105889,35.157466,2020,3,경상남도 진주시 가좌동 진주대로404번길,1,0,4.8,7750
1,경상남도 진주시 가좌동,1***,8,15.00,202003,11,500,40,2016,가좌길78번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길78번길,1,0,4.8,10500
5,경상남도 진주시 가좌동,5**,12,19.80,202003,14,200,36,2011,내동로348번길,128.106121,35.158214,2020,3,경상남도 진주시 가좌동 내동로348번길,1,0,4.8,9200
6,경상남도 진주시 가좌동,1***,8,20.00,202003,26,300,30,2016,가좌길48번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길48번길,1,0,4.8,7800
7,경상남도 진주시 가좌동,1***,12,20.10,202003,8,300,30,2014,진주대로404번길,128.105966,35.156961,2020,3,경상남도 진주시 가좌동 진주대로404번길,1,0,4.8,7800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166,경상남도 진주시 가좌동,1***,12,25.00,202011,28,5000,0,2017,진주대로404번길,128.105857,35.157800,2020,11,경상남도 진주시 가좌동 진주대로404번길,0,1,4.7,5000
3167,경상남도 진주시 가좌동,5**,12,30.00,202011,24,300,37,2007,진주대로500번길,128.108498,35.154904,2020,11,경상남도 진주시 가좌동 진주대로500번길,1,0,4.7,9747
3168,경상남도 진주시 가좌동,1***,8,30.00,202011,27,2000,20,2019,내동로320번길,128.107180,35.162495,2020,11,경상남도 진주시 가좌동 내동로320번길,1,0,4.7,7106
3169,경상남도 진주시 가좌동,1***,12,36.30,202011,26,300,55,1996,가좌길36번길,128.109160,35.157148,2020,11,경상남도 진주시 가좌동 가좌길36번길,1,0,4.7,14343


In [1042]:
corr = gajwa.corr()
corr

,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,경도,위도,year,month,전월세구분_월세,전월세구분_전세,전세전환가격
도로조건,1.000000,0.115301,-0.136298,-0.001694,-0.013106,0.083286,-0.155340,0.049289,-0.039431,-0.136197,-0.003794,0.031226,-0.031226,0.061130
계약면적(㎡),0.115301,1.000000,-0.201187,-0.006753,0.343500,0.116141,-0.093352,-0.026907,-0.039054,-0.202356,0.089829,-0.012119,0.012119,0.392502
계약년월,-0.136298,-0.201187,1.000000,-0.010998,-0.059855,-0.035536,0.142348,0.037678,-0.057461,0.999905,-0.018983,-0.027530,0.027530,0.240891
계약일,-0.001694,-0.006753,-0.010998,1.000000,0.015558,-0.010512,0.006402,-0.004358,-0.020172,-0.011956,0.069617,0.007043,-0.007043,0.001025
보증금(만원),-0.013106,0.343500,-0.059855,0.015558,1.000000,-0.725357,0.079807,0.025578,-0.032326,-0.061108,0.092254,-0.647643,0.647643,-0.134923
월세(만원),0.083286,0.116141,-0.035536,-0.010512,-0.725357,1.000000,-0.074669,-0.014294,0.013838,-0.034722,-0.057955,0.762797,-0.762797,0.710936
건축년도,-0.155340,-0.093352,0.142348,0.006402,0.079807,-0.074669,1.000000,0.008597,-0.009881,0.142690,-0.028468,-0.101014,0.101014,-0.001156
경도,0.049289,-0.026907,0.037678,-0.004358,0.025578,-0.014294,0.008597,1.000000,-0.521559,0.037759,-0.006813,-0.011087,0.011087,0.013954
위도,-0.039431,-0.039054,-0.057461,-0.020172,-0.032326,0.013838,-0.009881,-0.521559,1.000000,-0.057327,-0.008189,0.022232,-0.022232,-0.029077
year,-0.136197,-0.202356,0.999905,-0.011956,-0.061108,-0.034722,0.142690,0.037759,-0.057327,1.000000,-0.032791,-0.026964,0.026964,0.240234


In [1043]:
# # 전세 월세 구분 데이터셋
# gajwa = gajwa[gajwa.전월세구분 == "월세"]
# gajwa = pd.DataFrame.drop(gajwa, columns=["전월세구분"], axis=1)
# gajwa

## 지도

In [1044]:
import math as m

def distance(x1, x2, y1, y2):
    radius = 6371 # 지구 반지름(km)
    toRadian = m.pi / 180

    deltaLatitude = abs(x1 - x2) * toRadian
    deltaLongitude = abs(y1 - y2) * toRadian

    sinDeltaLat = m.sin(deltaLatitude / 2)
    sinDeltaLng = m.sin(deltaLongitude / 2)
    squareRoot = m.sqrt(sinDeltaLat * sinDeltaLat + m.cos(x1 * toRadian) * m.cos(x2 * toRadian) * sinDeltaLng * sinDeltaLng)

    distance = 2 * radius * m.asin(squareRoot)

    return distance

In [1045]:
gnu_long = 35.1530444
gnu_lat = 128.1010899

dist = []
for long, lat in zip(gajwa["경도"], gajwa["위도"]):
    d = distance(lat, long, gnu_lat, gnu_long)
    dist.append(d)

gajwa["distance"] = dist

In [1047]:
gajwa_data = pd.DataFrame.drop(gajwa, columns=["시군구","계약년월", "번지", "도로명", "계약일", "보증금(만원)", "월세(만원)", "location", "경도", "위도", "전월세전환율"], axis=1)

#gajwa_data.distance = gajwa_data.distance.transform(lambda x: (x - x.mean()) / x.std())
#gajwa_data.전세전환가격 = gajwa_data.전세전환가격.transform(lambda x: (x - x.mean()) / x.std())
gajwa_data

,도로조건,계약면적(㎡),건축년도,year,month,전월세구분_월세,전월세구분_전세,전세전환가격,distance
0,12,12.00,2007,2020,3,1,0,7750,6824.774167
1,8,15.00,2016,2020,3,1,0,10500,6824.746434
5,12,19.80,2011,2020,3,1,0,9200,6824.724392
6,8,20.00,2016,2020,3,1,0,7800,6824.746434
7,12,20.10,2014,2020,3,1,0,7800,6824.817255
...,...,...,...,...,...,...,...,...,...
3166,12,25.00,2017,2020,11,0,1,5000,6824.746434
3167,12,30.00,2007,2020,11,1,0,9747,6825.083558
3168,8,30.00,2019,2020,11,1,0,7106,6824.428700
3169,12,36.30,1996,2020,11,1,0,14343,6824.932898


# 머신러닝

In [1048]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
import lightgbm as lgb
import catboost
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [1049]:
X = gajwa_data.drop(["전세전환가격"], axis=1)
y = gajwa_data['전세전환가격']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1645)

scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.fit_transform(X_test)

minmaxScaler = MinMaxScaler()
X_train_standardized2 = minmaxScaler.fit_transform(X_train)
X_test_standardized2 = minmaxScaler.fit_transform(X_test)

## 선형회귀분석

In [1050]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)
# y_preds = lr.predict(X_test)

# mse = mean_squared_error(y_test, y_preds)
# rmse = np.sqrt(mse)

# print(f"MSE : {mse:.3f}", f"RMSE: {rmse:.3f}")
# print(f"variance score : {r2_score(y_test, y_preds):.3f}")

In [1051]:
# lr.fit(X_train_standardized, y_train)
# lr.score(X_test_standardized, y_test)

## Ridge

In [1052]:
# R_model = Ridge()
# R_model.fit(X_train, y_train)
# print(R_model.score(X_test, y_test))

# R_model.fit(X_train_standardized, y_train)
# R_model.score(X_test_standardized, y_test)

## Lasso

In [1053]:
# L_model = Lasso(alpha=0.5)
# L_model.fit(X_train, y_train)
# print(L_model.score(X_test, y_test))

# L_model.fit(X_train_standardized, y_train)
# L_model.score(X_test_standardized, y_test)

## ElasticNet

In [1054]:
# ela = ElasticNet(alpha=0.5)
# ela.fit(X_train, y_train)
# print(ela.score(X_test, y_test))

# ela.fit(X_train_standardized, y_train)
# ela.score(X_test_standardized, y_test)

## 랜덤포레스트

In [1055]:
randomforest = RandomForestRegressor(random_state=0, n_jobs=-1)
randomforest_model = randomforest.fit(X_train, y_train)
print(randomforest_model.score(X_test, y_test))

randomforest_model = randomforest.fit(X_train_standardized, y_train)
print(randomforest_model.score(X_test_standardized, y_test))

0.6836688137615506
0.6855588111442016


In [1056]:
gbm = GradientBoostingRegressor()

gbm.fit(X_train, y_train)
print(gbm.score(X_test, y_test))
gbm.fit(X_train_standardized, y_train)
print(gbm.score(X_test_standardized, y_test))

0.6907961101180271
0.6947637792734516


## LightGBM

In [1057]:
lightgbm = lgb.LGBMRegressor()

lightgbm.fit(X_train, y_train)
print(lightgbm.score(X_test, y_test))

lightgbm.fit(X_train_standardized, y_train)
print(lightgbm.score(X_test_standardized, y_test))

0.6918894258457273
0.6815535810142086


## XGBoost

In [1025]:
xgboost = XGBRegressor(n_estimators=50)
xgboost.fit(X_train, y_train)
print(xgboost.score(X_test, y_test))

xgboost.fit(X_train_standardized, y_train)
xgboost.score(X_test_standardized, y_test)

0.6392934494924998


/Users/chance/opt/anaconda3/envs/py3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.6600311378897044

## CatBoost

In [1026]:
cb = catboost.CatBoostRegressor()
cb.fit(X_train_standardized, y_train)
cb.score(X_test_standardized, y_test)

Learning rate set to 0.046193
0:	learn: 2809.1161038	total: 1.82ms	remaining: 1.81s
1:	learn: 2746.8743016	total: 2.56ms	remaining: 1.28s
2:	learn: 2685.8710646	total: 3.26ms	remaining: 1.08s
3:	learn: 2626.1791260	total: 4.03ms	remaining: 1s
4:	learn: 2569.5449200	total: 4.76ms	remaining: 948ms
5:	learn: 2517.2662420	total: 5.38ms	remaining: 891ms
6:	learn: 2468.4723423	total: 5.76ms	remaining: 817ms
7:	learn: 2425.7221885	total: 6.5ms	remaining: 805ms
8:	learn: 2380.8746054	total: 7.02ms	remaining: 773ms
9:	learn: 2339.5037351	total: 7.51ms	remaining: 743ms
10:	learn: 2301.9151044	total: 8.1ms	remaining: 728ms
11:	learn: 2262.5278813	total: 8.78ms	remaining: 723ms
12:	learn: 2228.8081114	total: 9.37ms	remaining: 711ms
13:	learn: 2197.3895975	total: 9.9ms	remaining: 697ms
14:	learn: 2165.3344076	total: 10.5ms	remaining: 687ms
15:	learn: 2136.3568655	total: 11.1ms	remaining: 685ms
16:	learn: 2110.6486433	total: 11.9ms	remaining: 686ms
17:	learn: 2086.4043624	total: 12.7ms	remaining: 69

0.6873933359892512